In [15]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm_notebook
from new_utils import SolrManager
from new_utils import OhdsiManager
from new_utils import IdManager
import datetime

In [3]:
class preliminary_analysis():
    def __init__(self, df):
        self.df = df
    
    def get_info_raw(self):
        '''Get the shape, number of unique patients, patient types, etc'''
        print(f"The shape of the data is {self.df.shape}")

        print("--------------------")

        print(f"The number of patients in total: {self.df['Epic MRN'].nunique()}")

        print("--------------------")
        
        print(self.df['label'].value_counts())
        print("--------------------")
        print(self.df["Type of patient"].value_counts())



class preprocessing():
    def __init__(self, df):
        self.df = df
    
    def get_all_notes_from_MRN(self):
        solr_note = SolrManager()
        df_notes = pd.DataFrame()
        df_exceptions_dict = {"emp": []}
        MRN_list = self.df["Epic MRN"].copy()
        for i in tqdm_notebook(range(len(MRN_list))):
            note = solr_note.get_note(MRN_list.iloc[i])
            if note is None:
                df_exceptions_dict["emp"].append(MRN_list.iloc[i])
                continue
            # except KeyError:
            #     df_exceptions_dict["emp"].append(MRN_list.iloc[i])
            #     continue
            df_notes = pd.concat([df_notes, note],axis=0)

        self.df = self.df.merge(df_notes,left_on="Epic MRN", right_on="empi")
        self.df.drop(columns=["empi"], inplace=True)
        return self.df, df_exceptions_dict
    
    def maping_to_ids(self):
        list_epic = list(df_whole["Epic MRN"].copy())
        id_mapping = IdManager(type='epic')
        id_mapping.addIdList(list_epic)
        id_mapping.getAllIds()
        id_mapping.IdMappingDf["EMPI"] = pd.to_numeric(id_mapping.IdMappingDf["EMPI"])
        print("Number of patients linked to EHR")
        print(id_mapping.IdMappingDf["person_id"].nunique())
        print(id_mapping.IdMappingDf.dtypes)
        self.df = self.df.merge(id_mapping.IdMappingDf, left_on="Epic MRN", right_on = "EMPI")
        
        return self.df

class get_structure():
    def __init__(self, df):
        self.df = df
    
    def get_demographics(self):
        pid = tuple(set(self.df["person_id"]))
        where_clause = f"WHERE p.person_id in {pid}"
        print(where_clause)
        sql_query = f'''SELECT p.person_id, p.birth_datetime, 
                       p.ethnicity_source_value, p.gender_source_value, p.race_source_value
                       FROM dbo.person p
                       {where_clause}
                       '''
        connector = OhdsiManager()
        demograhocs_df = connector.get_dataFromQuery(sql_query)
        return demograhocs_df
    


In [4]:
## Load Notes & Select columns f
df_WES = pd.read_excel("datasets/WES vs panel.xlsx", 
                       sheet_name="WES-WGS")
df_panel = pd.read_excel("datasets/WES vs panel.xlsx",
                         sheet_name="Panel")
df_panel.rename(columns={"Other features":"If other, specify:"}, inplace=True)
df_panel_WES = pd.read_excel("datasets/WES vs panel.xlsx",
                             sheet_name="Tiered-panel-WES")

df_panel_WES["label"] = "WES"
df_WES["label"] = "WES"
df_panel["label"] = "panel"


df_WES  = df_WES[["Epic MRN", "Primary indication", "If other, specify:", "Type of patient", "label"]]
df_panel = df_panel[["Epic MRN","Primary indication", "If other, specify:", "Type of patient","label"]]
df_panel_WES = df_panel_WES[["Epic MRN", "Primary indication", "If other, specify:", "Type of patient", "label"]]

df_whole = pd.concat([df_WES, df_panel, df_panel_WES], axis=0)

In [5]:
# Drop duplicates
df_whole = df_whole.drop_duplicates()
df_whole.reset_index(drop=True, inplace=True)

# Remove invalid Epic MRN & Irrelevant
# invalid_mrn list refer to data preprocessing.docx from the one-drive genetic testing project
# invalid_mrn = [] 
drop_idx = []
for mrn in invalid_mrn:
    drop_idx.append(df_whole[df_whole["Epic MRN"] == mrn].index.values[0])
df_whole.drop(index=drop_idx, inplace=True)
df_whole.reset_index(drop=True, inplace=True)


In [6]:
print(f"Final df shape {df_whole.shape}")
preliminary_analyzer = preliminary_analysis(df_whole)
preliminary_analyzer.get_info_raw()

Final df shape (1199, 5)
The shape of the data is (1199, 5)
--------------------
The number of patients in total: 1199
--------------------
label
WES      651
panel    548
Name: count, dtype: int64
--------------------
Type of patient
Peds OP    919
Peds IP    280
Name: count, dtype: int64


In [7]:
# Acquire all notes from chart
preprocessor = preprocessing(df_whole)
df_WithNotes, df_exceptions_dict = preprocessor.get_all_notes_from_MRN()
# 417 unique patients 

  0%|          | 0/1199 [00:00<?, ?it/s]

In [11]:
df_exceptions_d  = pd.DataFrame(df_exceptions_dict)
df_exceptions_dict.to_csv("patients_Without_notes.csv", index=False)

In [95]:
df_demographics.to_csv("exported_data/pt_demographics.csv", index=False)
df_WithNotes.to_csv("exported_data/df_withNotes.csv", index=False)